In [1]:
from weiner_variation.sim.data_structures import DrawInput
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm
from copy import deepcopy
import tqdm
from multiprocessing import Pool

In [2]:
import pyroll.basic as pr
import pyroll.jmak
import pyroll.elastic_mill_spring

@pr.RollPass.stand_stiffness
def stiffness(self):
    return 2.3e8

In [3]:
nominal = DrawInput(DIAMETER, TEMPERATURE)

In [4]:
input_dist = pd.read_csv(DATA_DIR / "input_dist.csv", index_col=0, header=0)
input_dist

,mean,std
property,,
diameter,0.05,0.001
temperature,1373.15,10.000


In [5]:
diameter_dist = norm(loc=input_dist.loc["diameter", "mean"], scale=input_dist.loc["diameter", "std"])
temperature_dist = norm(loc=input_dist.loc["temperature", "mean"], scale=input_dist.loc["temperature", "std"])

In [6]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)

draws = [nominal] + [
    DrawInput(d, t)
    for d, t in zip(diameters, temperatures)
]

In [7]:
def worker(draw: DrawInput):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [8]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

501it [00:12, 39.44it/s]                                                                                                                                                                                                                                                                                                                                                                                                                                                            


In [9]:
df = results |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                  roll_force                                \
     diameter  temperature             R1             R2             R3   
0    0.050000  1373.150000  304773.089762  179799.435302  303242.645910   
1    0.050390  1377.628215  306513.577326  180280.401782  301465.517731   
2    0.050772  1362.139658  325540.903771  190188.561066  313725.125332   
3    0.051325  1382.460744  314400.156336  184060.869654  301255.240757   
4    0.049996  1375.708412  302509.434659  178594.581195  301448.567737   
..        ...          ...            ...            ...            ...   
496  0.049818  1375.215264  300534.742665  177648.653531  301168.987755   
497  0.049424  1368.666618  300779.381079  178005.862587  304367.097798   
498  0.051160  1350.303449  342170.306573  198756.829399  323745.089383   
499  0.050589  1366.183130  319334.861866  186992.583276  310190.901342   
500  0.051031  1353.790165  336992.580825  196097.457464  320711.678246   

                                                                 \
                R4             R5             R6             R7   
0    141247.950228  206695.584512  128251.173705  233233.204303   
1    139988.595153  205059.564456  127377.717627  231876.087493   
2    144319.489158  210532.576295  130200.665972  236215.272691   
3    138644.628184  203251.781082  126374.682795  230328.265961   
4    140536.103252  205789.521654  127780.373771  232507.405191   
..             ...            ...            ...            ...   
496  140677.968108  205988.493588  127892.785444  232685.371263   
497  142532.597923  208379.917336  129163.430929  234654.789387   
498  147714.204385  214796.520499  132391.360952  239570.846180   
499  143179.305841  209100.963518  129467.416016  235091.594899   
500  146706.367560  213536.131814  131743.171149  238579.013931   

                    ...    strain                                          \
                R8  ...        R5        R6        R7        R8        R9   
0    123374.184883  ...  0.595997  0.499029  0.727971  0.596275  0.458519   
1    122812.228882  ...  0.596084  0.499048  0.727972  0.596263  0.458509   
2    124595.899010  ...  0.596202  0.499127  0.728015  0.596317  0.458546   
3    122173.758967  ...  0.596512  0.499030  0.727960  0.596244  0.458496   
4    123075.154227  ...  0.595991  0.499021  0.727966  0.596266  0.458513   
..             ...  ...       ...       ...       ...       ...       ...   
496  123149.598336  ...  0.595947  0.499007  0.727962  0.596267  0.458514   
497  123963.575475  ...  0.595855  0.498991  0.727965  0.596286  0.458528   
498  125970.011560  ...  0.596309  0.499194  0.728051  0.596360  0.458575   
499  124135.188917  ...  0.596152  0.499100  0.728002  0.596302  0.458536   
500  125564.230628  ...  0.596275  0.499173  0.728040  0.596347  0.458567   

                                                       
          R10        F1        F2        F3        F4  
0    0.460103  0.335068  0.360713  0.456285  0.311359  
1    0.460089  0.335060  0.360704  0.456281  0.311353  
2    0.460137  0.335086  0.360733  0.456294  0.311373  
3    0.460072  0.335051  0.360693  0.456277  0.311346  
4    0.460095  0.335064  0.360708  0.456283  0.311356  
..        ...       ...       ...       ...       ...  
496  0.460097  0.335065  0.360709  0.456284  0.311357  
497  0.460118  0.335076  0.360722  0.456289  0.311366  
498  0.460174  0.335107  0.360755  0.456303  0.311388  
499  0.460124  0.335080  0.360725  0.456291  0.311368  
500  0.460163  0.335101  0.360748  0.456301  0.311383  

[501 rows x 100 columns]

In [10]:
df.to_csv(DATA_DIR / "sim_elastic_results.csv")